<a href="https://www.kaggle.com/code/beltagymohamedsaleh/audible-cleaning?scriptVersionId=196552044" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Data & Libraries

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/kaggle/input/audible-dataset/audible_uncleaned.csv")

# Data Exploratory & Cleaning

In [3]:
df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


### found that all columns are of type object. I will clean and convert relevant columns to appropriate types (e.g., date, numeric) and extract useful information from stars and time.




##  Clean the 'author' and 'narrator' columns by removing specific prefixes and trimming extra spaces.
## The 'replace' method is used to remove 'Writtenby:' from the 'author' column and 'Narratedby:' from the 'narrator' column.
## The 'strip' method removes any leading or trailing whitespace.

In [5]:
df['Author'] = df['author'].str.replace('Writtenby:', '', regex=False).str.strip() #replace & regular expressions & delete any spaces
df['Narrator'] = df['narrator'].str.replace('Narratedby:', '', regex=False).str.strip()

In [6]:
df.head()

,name,author,narrator,time,releasedate,language,stars,price,Author,Narrator
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00,GeronimoStilton,BillLobely
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00,RickRiordan,RobbieDaymond
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00,JeffKinney,DanRussell
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00,RickRiordan,SoneelaNankani
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00,RickRiordan,JesseBernstein


## Drop the old columns "author" and "narrator".

In [7]:
df.drop(["author", "narrator"], axis=1, inplace=True)

In [8]:
df.columns

Index(['name', 'time', 'releasedate', 'language', 'stars', 'price', 'Author',
       'Narrator'],
      dtype='object')

In [9]:
df.shape

(87489, 8)

In [10]:
df.isnull().sum()

name           0
time           0
releasedate    0
language       0
stars          0
price          0
Author         0
Narrator       0
dtype: int64

## Clean the 'stars' column by removing the text 'out of 5 stars' and stripping any extra spaces.
## The 'replace' method removes the specified text, and 'strip' ensures there are no leading or trailing whitespace.


In [11]:
df['stars'] = df['stars'].str.replace('out of 5 stars', '', regex=False).str.strip()

## Extract the numeric rating from the 'stars' column by splitting the string on whitespace and selecting the first element.
## This assumes the rating is the first part of the split result.

In [12]:
df['stars'] = df['stars'].str.split().str[0]

In [13]:
df.columns

Index(['name', 'time', 'releasedate', 'language', 'stars', 'price', 'Author',
       'Narrator'],
      dtype='object')

## Convert 'price' column to numeric format for consistent analysis.
## Replace 'Free' with '0' to represent free items as zero, then remove any commas from the 'price' values.
## Convert the series to a string type for cleaning, and then to float for numerical analysis.

In [14]:
df['Price'] = df['price'].replace('Free', '0').astype(str).str.replace(',', '') # Convert the series to a string type before using the .str accessor
df['Price'] = df['Price'].astype(float) # Convert the 'Price' column to float

## Convert the 'releasedate' column to datetime format for easier date manipulation and analysis.
## The 'format' parameter specifies that the date format is day-month-year (e.g., '31-12-23').

In [15]:
df['Releasedate'] = pd.to_datetime(df['releasedate'], format='%d-%m-%y')

## Define a function to convert time strings into total minutes.
## The function splits the time string into hours and minutes, converts them to integers, and calculates the total minutes.
## The 'apply' method is then used to apply this function to the 'time' column and create a new column 'Time_in_Minutes'.

In [16]:
def convert_time_to_minutes(time_str):
    time_parts = time_str.split(' and ')
    hours = int(time_parts[0].split(' hrs')[0]) if 'hrs' in time_parts[0] else 0
    minutes = int(time_parts[1].split(' mins')[0]) if len(time_parts) > 1 and 'mins' in time_parts[1] else 0
    return hours * 60 + minutes

# Check for typos and correct the column name
df['Time_in_Minutes'] = df['time'].apply(convert_time_to_minutes)

In [17]:
df


,name,time,releasedate,language,stars,price,Author,Narrator,Price,Releasedate,Time_in_Minutes
0,Geronimo Stilton #11 & #12,2 hrs and 20 mins,04-08-08,English,5,468.00,GeronimoStilton,BillLobely,468.0,2008-08-04,140
1,The Burning Maze,13 hrs and 8 mins,01-05-18,English,4.5,820.00,RickRiordan,RobbieDaymond,820.0,2018-05-01,788
2,The Deep End,2 hrs and 3 mins,06-11-20,English,4.5,410.00,JeffKinney,DanRussell,410.0,2020-11-06,123
3,Daughter of the Deep,11 hrs and 16 mins,05-10-21,English,4.5,615.00,RickRiordan,SoneelaNankani,615.0,2021-10-05,676
4,"The Lightning Thief: Percy Jackson, Book 1",10 hrs,13-01-10,English,4.5,820.00,RickRiordan,JesseBernstein,820.0,2010-01-13,600
...,...,...,...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,7 hrs and 34 mins,09-03-17,English,Not,596.00,ChrisStewart,ChrisStewart,596.0,2017-03-09,454
87485,The Alps,10 hrs and 7 mins,21-02-17,English,Not,820.00,StephenO'Shea,RobertFass,820.0,2017-02-21,607
87486,The Innocents Abroad,19 hrs and 4 mins,30-12-16,English,Not,938.00,MarkTwain,FloGibson,938.0,2016-12-30,1144
87487,A Sentimental Journey,4 hrs and 8 mins,23-02-11,English,Not,680.00,LaurenceSterne,AntonLesser,680.0,2011-02-23,248


## Remove the 'time' column from the DataFrame as it is no longer needed after conversion to minutes.
## The 'axis=1' parameter specifies that we are dropping a column, and 'inplace=True' updates the DataFrame in place.

In [18]:
df.drop(["time"],axis=1,inplace=True)

In [19]:
df

,name,releasedate,language,stars,price,Author,Narrator,Price,Releasedate,Time_in_Minutes
0,Geronimo Stilton #11 & #12,04-08-08,English,5,468.00,GeronimoStilton,BillLobely,468.0,2008-08-04,140
1,The Burning Maze,01-05-18,English,4.5,820.00,RickRiordan,RobbieDaymond,820.0,2018-05-01,788
2,The Deep End,06-11-20,English,4.5,410.00,JeffKinney,DanRussell,410.0,2020-11-06,123
3,Daughter of the Deep,05-10-21,English,4.5,615.00,RickRiordan,SoneelaNankani,615.0,2021-10-05,676
4,"The Lightning Thief: Percy Jackson, Book 1",13-01-10,English,4.5,820.00,RickRiordan,JesseBernstein,820.0,2010-01-13,600
...,...,...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,09-03-17,English,Not,596.00,ChrisStewart,ChrisStewart,596.0,2017-03-09,454
87485,The Alps,21-02-17,English,Not,820.00,StephenO'Shea,RobertFass,820.0,2017-02-21,607
87486,The Innocents Abroad,30-12-16,English,Not,938.00,MarkTwain,FloGibson,938.0,2016-12-30,1144
87487,A Sentimental Journey,23-02-11,English,Not,680.00,LaurenceSterne,AntonLesser,680.0,2011-02-23,248


## Add a space between lowercase and uppercase letters in the 'Author' and 'Narrator' columns to improve readability.
## The regular expression r'([a-z])([A-Z])' identifies lowercase letters followed by uppercase letters and inserts a space between them.
## The 'regex=True' parameter ensures that the replacement is done using regular expressions.

In [20]:
df['Author'] = df['Author'].str.replace(r'([a-z])([A-Z])', r'\1 \2', regex=True)
df['Narrator'] = df['Narrator'].str.replace(r'([a-z])([A-Z])', r'\1 \2',regex=True)

print(df['Author'].head())

0    Geronimo Stilton
1        Rick Riordan
2         Jeff Kinney
3        Rick Riordan
4        Rick Riordan
Name: Author, dtype: object


In [21]:
df

,name,releasedate,language,stars,price,Author,Narrator,Price,Releasedate,Time_in_Minutes
0,Geronimo Stilton #11 & #12,04-08-08,English,5,468.00,Geronimo Stilton,Bill Lobely,468.0,2008-08-04,140
1,The Burning Maze,01-05-18,English,4.5,820.00,Rick Riordan,Robbie Daymond,820.0,2018-05-01,788
2,The Deep End,06-11-20,English,4.5,410.00,Jeff Kinney,Dan Russell,410.0,2020-11-06,123
3,Daughter of the Deep,05-10-21,English,4.5,615.00,Rick Riordan,Soneela Nankani,615.0,2021-10-05,676
4,"The Lightning Thief: Percy Jackson, Book 1",13-01-10,English,4.5,820.00,Rick Riordan,Jesse Bernstein,820.0,2010-01-13,600
...,...,...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,09-03-17,English,Not,596.00,Chris Stewart,Chris Stewart,596.0,2017-03-09,454
87485,The Alps,21-02-17,English,Not,820.00,Stephen O'Shea,Robert Fass,820.0,2017-02-21,607
87486,The Innocents Abroad,30-12-16,English,Not,938.00,Mark Twain,Flo Gibson,938.0,2016-12-30,1144
87487,A Sentimental Journey,23-02-11,English,Not,680.00,Laurence Sterne,Anton Lesser,680.0,2011-02-23,248


## Remove the 'price' and 'releasedate' columns from the DataFrame as they are no longer needed for analysis after we add "Price"	"Releasedate".
## The 'axis=1' parameter specifies that we are dropping columns, and 'inplace=True' updates the DataFrame directly.

In [22]:
df.drop(["price"], axis=1, inplace=True)
df.drop(['releasedate'], axis=1,inplace=True)

## Filter out rows where the 'stars' column has the value 'Not', as these entries are not useful for analysis.
## Reset the index after filtering to maintain a continuous index sequence.

In [23]:
df = df[df['stars'] != 'Not'].reset_index(drop=True)

In [24]:
df.isna().sum()
df.isnull().sum()

name               0
language           0
stars              0
Author             0
Narrator           0
Price              0
Releasedate        0
Time_in_Minutes    0
dtype: int64

# Display the number of missing values for each column in the DataFrame.
# This helps identify if any columns need further cleaning or imputation before analysis.


In [25]:
df

,name,language,stars,Author,Narrator,Price,Releasedate,Time_in_Minutes
0,Geronimo Stilton #11 & #12,English,5,Geronimo Stilton,Bill Lobely,468.0,2008-08-04,140
1,The Burning Maze,English,4.5,Rick Riordan,Robbie Daymond,820.0,2018-05-01,788
2,The Deep End,English,4.5,Jeff Kinney,Dan Russell,410.0,2020-11-06,123
3,Daughter of the Deep,English,4.5,Rick Riordan,Soneela Nankani,615.0,2021-10-05,676
4,"The Lightning Thief: Percy Jackson, Book 1",English,4.5,Rick Riordan,Jesse Bernstein,820.0,2010-01-13,600
...,...,...,...,...,...,...,...,...
15067,Why We Fly,English,5,Evan Rail,Evan Rail,100.0,2014-06-23,0
15068,River Town,English,5,Peter Hessler,Peter Berkrot,836.0,2010-05-04,876
15069,Figures in a Landscape,English,4,Paul Theroux,Edoardo Ballerini,949.0,2018-06-07,1004
15070,Tower of London,English,5,i Minds,Abbey Holmes,33.0,2009-12-23,0


# Thank You